# **UTS**

Lakukan klasifikasi teks abstrak dengan Naive Bayes/KNN menggunakan representasi test TF-IDF dengan reduksi dimensi PCA

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


1. Load Data

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/Nicolas271101/prosaindata-/main/dataset-baru.csv")
df

,Unnamed: 0,Abstrak,Kategori
0,0,Sistem informasi akademik (SIAKAD) merupaka...,RPL
1,1,Berjalannya koneksi jaringan komputer dengan l...,RPL
2,2,Web server adalah sebuah perangkat lunak serve...,RPL
3,3,Penjadwalan kuliah di Perguruan Tinggi me...,KOMPUTASI
4,4,Seiring perkembangan teknologi yang ada diduni...,RPL
...,...,...,...
800,804,Investasi saham selama ini memiliki resiko ker...,KOMPUTASI
801,805,Information Retrieval (IR) merupakan pengambil...,KOMPUTASI
802,806,Klasifikasi citra merupakan proses pengelompok...,KOMPUTASI
803,807,Identifikasi atribut pejalan kaki merupakan sa...,KOMPUTASI


In [ ]:
text = df['Abstrak']
Y = df.values[:, -1]
#Y
text

0      sistem informasi akademik siakad sistem inform...
1      berjalannya koneksi jaringan komputer lancar g...
2      web server perangkat lunak server berfungsi me...
3      penjadwalan kuliah perguruan kompleks permasal...
4      seiring perkembangan teknologi didunia muncul ...
                             ...                        
800    investasi saham memiliki resiko kerugian perge...
801    information retrieval ir pengambilan informasi...
802    klasifikasi citra proses pengelompokan piksel ...
803    identifikasi atribut pejalan kaki salah peneli...
804    topik deteksi objek menarik perhatian perkemba...
Name: Abstrak, Length: 805, dtype: object

In [ ]:
df["Abstrak"] = df['Abstrak'].str.lower()
df["Abstrak"]

0      sistem informasi akademik siakad sistem inform...
1      berjalannya koneksi jaringan komputer lancar g...
2      web server perangkat lunak server berfungsi me...
3      penjadwalan kuliah perguruan kompleks permasal...
4      seiring perkembangan teknologi didunia muncul ...
                             ...                        
800    investasi saham memiliki resiko kerugian perge...
801    information retrieval ir pengambilan informasi...
802    klasifikasi citra proses pengelompokan piksel ...
803    identifikasi atribut pejalan kaki salah peneli...
804    topik deteksi objek menarik perhatian perkemba...
Name: Abstrak, Length: 805, dtype: object

In [ ]:
#cleaning
import re
clean =[]
for i in range (len(df["Abstrak"])): 
  clean_tag  = re.sub("@[A-Za-z0-9_]+","", df["Abstrak"][i]) #clenasing mention
  clean_hashtag = re.sub("#[A-Za-z0-9_]+","", clean_tag) #clenasing hashtag 
  clean_https = re.sub(r'http\S+', '', clean_hashtag) #cleansing url link
  clean_symbols = re.sub("[^a-zA-Zï ]+"," ", clean_https) #cleansing character

  clean.append(clean_symbols)

  
df["Abstrak"] = clean
df["Abstrak"]

0      sistem informasi akademik siakad sistem inform...
1      berjalannya koneksi jaringan komputer lancar g...
2      web server perangkat lunak server berfungsi me...
3      penjadwalan kuliah perguruan kompleks permasal...
4      seiring perkembangan teknologi didunia muncul ...
                             ...                        
800    investasi saham memiliki resiko kerugian perge...
801    information retrieval ir pengambilan informasi...
802    klasifikasi citra proses pengelompokan piksel ...
803    identifikasi atribut pejalan kaki salah peneli...
804    topik deteksi objek menarik perhatian perkemba...
Name: Abstrak, Length: 805, dtype: object

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Membuat kamus slang words dan kata Indonesia yang benar
slang_dict = pd.read_csv("https://raw.githubusercontent.com/Nicolas271101/prosaindata-/main/combined_slang_words.txt", sep=" ", header=None)

# Membuat fungsi untuk mengubah slang words menjadi kata Indonesia yang benar
def replace_slang_words(text):
    words = nltk.word_tokenize(text.lower())
    words_filtered = [word for word in words if word not in stopwords.words('indonesian')]
    for i in range(len(words_filtered)):
        if words_filtered[i] in slang_dict:
            words_filtered[i] = slang_dict[words_filtered[i]]
    return ' '.join(words_filtered)

# Contoh penggunaan

slang_words=[]
for i in range(len(clean)):
  slang = replace_slang_words(clean[i])
  slang_words.append(slang)

df["Abstrak"] = slang_words
df["Abstrak"]

0      sistem informasi akademik siakad sistem inform...
1      berjalannya koneksi jaringan komputer lancar g...
2      web server perangkat lunak server berfungsi me...
3      penjadwalan kuliah perguruan kompleks permasal...
4      seiring perkembangan teknologi didunia muncul ...
                             ...                        
800    investasi saham memiliki resiko kerugian perge...
801    information retrieval ir pengambilan informasi...
802    klasifikasi citra proses pengelompokan piksel ...
803    identifikasi atribut pejalan kaki salah peneli...
804    topik deteksi objek menarik perhatian perkemba...
Name: Abstrak, Length: 805, dtype: object

In [ ]:
#proses tokenisasi
# from nltk.tokenize import TweetTokenizer
# def word_tokenize(text):
#   tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
#   return tokenizer.tokenize(text)

df["Abstrak"]= df["Abstrak"].apply(lambda sentence: nltk.word_tokenize(sentence))
df["Abstrak"]

0      [sistem, informasi, akademik, siakad, sistem, ...
1      [berjalannya, koneksi, jaringan, komputer, lan...
2      [web, server, perangkat, lunak, server, berfun...
3      [penjadwalan, kuliah, perguruan, kompleks, per...
4      [seiring, perkembangan, teknologi, didunia, mu...
                             ...                        
800    [investasi, saham, memiliki, resiko, kerugian,...
801    [information, retrieval, ir, pengambilan, info...
802    [klasifikasi, citra, proses, pengelompokan, pi...
803    [identifikasi, atribut, pejalan, kaki, salah, ...
804    [topik, deteksi, objek, menarik, perhatian, pe...
Name: Abstrak, Length: 805, dtype: object

In [ ]:
txt_stopwords = stopwords.words('indonesian')

def stopwords_removal(filtering) :
  filtering = [word for word in filtering if word not in txt_stopwords]
  return filtering

df["Abstrak"] = df["Abstrak"].apply(stopwords_removal)
df["Abstrak"]

0      [sistem, informasi, akademik, siakad, sistem, ...
1      [berjalannya, koneksi, jaringan, komputer, lan...
2      [web, server, perangkat, lunak, server, berfun...
3      [penjadwalan, kuliah, perguruan, kompleks, per...
4      [seiring, perkembangan, teknologi, didunia, mu...
                             ...                        
800    [investasi, saham, memiliki, resiko, kerugian,...
801    [information, retrieval, ir, pengambilan, info...
802    [klasifikasi, citra, proses, pengelompokan, pi...
803    [identifikasi, atribut, pejalan, kaki, salah, ...
804    [topik, deteksi, objek, menarik, perhatian, pe...
Name: Abstrak, Length: 805, dtype: object

In [ ]:
!pip install sastrawi
!pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#proses stem
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import string
import swifter
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming (term):
  return stemmer.stem(term)

term_dict = {}
for document in df["Abstrak"]:
  for term in document:
    if term not in term_dict:
      term_dict[term] = ''


In [ ]:
term_dict

{'sistem': '',
 'informasi': '',
 'akademik': '',
 'siakad': '',
 'berfungsi': '',
 'menangani': '',
 'pengelolaan': '',
 'penyajian': '',
 'data': '',
 'fakultas': '',
 'dianggap': '',
 'pelayanan': '',
 'mahasiswa': '',
 'membutuhkan': '',
 'universitas': '',
 'trunojoyo': '',
 'tersedia': '',
 'database': '',
 'terpusat': '',
 'kelebihan': '',
 'perawatannya': '',
 'mudah': '',
 'biaya': '',
 'berpotensi': '',
 'mengahadapi': '',
 'kendala': '',
 'proses': '',
 'transaksi': '',
 'padatnya': '',
 'jaringan': '',
 'kelambatan': '',
 'pemrosesan': '',
 'respon': '',
 'query': '',
 'tersimpan': '',
 'kompleks': '',
 'memiliki': '',
 'kelemahan': '',
 'ketersediaan': '',
 'pengembangan': '',
 'databases': '',
 'terdistribusi': '',
 'dijadikan': '',
 'solusi': '',
 'permasalahan': '',
 'basisdata': '',
 'keuntungan': '',
 'dimiliki': '',
 'pengawasan': '',
 'distribusi': '',
 'reability': '',
 'availability': '',
 'kecepatan': '',
 'otonomi': '',
 'local': '',
 'berjalannya': '',
 'koneks

In [ ]:
print(len(term_dict))
print("-----------------------------")

8147
-----------------------------


In [ ]:
for term in term_dict:
  term_dict[term] = stemming(term)
  print(term,":",term_dict[term])

print(term_dict)
print("-----------------------------")

Output streaming akan dipotong hingga 5000 baris terakhir.
unas : unas
melepaskan : lepas
lepas : lepas
perumahan : rumah
permukiman : mukim
rtlh : rtlh
kemenpera : kemenpera
deputi : deputi
stimulan : stimulan
swadaya : swadaya
bsps : bsps
berpenghasilan : hasil
meberikan : meberikan
pramuka : pramuka
bendera : bendera
semaphore : semaphore
adobe : adobe
macromedia : macromedia
flag : flag
signaling : signaling
os : os
sandi : sandi
step : step
berurutan : rurut
praktek : praktek
mengetikkan : etik
textbox : textbox
diketikkan : ketik
disuguhi : suguh
anggota : anggota
tespilihan : tespilihan
ganda : ganda
diteskan : tes
diatasi : atas
esai : esai
objektif : objektif
biologi : biologi
sman : sman
ploso : ploso
pengecekan : kece
sinonim : sinonim
ecsp : ecsp
mencontek : contek
membandigkan : membandigkan
jombang : jombang
mengoreksi : koreksi
rmse : rmse
tulang : tulang
winnowingadalah : winnowingadalah
jaccard : jaccard
coeficient : coeficient
menunjukkanpenggunaan : menunjukkanpenggu

In [ ]:
def get_stemming(document):
  return [term_dict[term] for term in document]

In [ ]:
df['Abstrak'] = df['Abstrak'].swifter.apply(get_stemming)
df['Abstrak'] 

Pandas Apply:   0%|          | 0/805 [00:00<?, ?it/s]

0      [sistem, informasi, akademik, siakad, sistem, ...
1      [jalan, koneksi, jaring, komputer, lancar, gan...
2      [web, server, perangkat, lunak, server, fungsi...
3      [jadwal, kuliah, guru, kompleks, masalah, vari...
4      [iring, kembang, teknologi, dunia, muncul, tek...
                             ...                        
800    [investasi, saham, milik, resiko, rugi, gera, ...
801    [information, retrieval, ir, ambil, informasi,...
802    [klasifikasi, citra, proses, kelompok, piksel,...
803    [identifikasi, atribut, pejal, kaki, salah, te...
804    [topik, deteksi, objek, tarik, perhati, kemban...
Name: Abstrak, Length: 805, dtype: object

In [ ]:
def join_kata(data):
  kalimat = ''
  for i in data:
    kalimat += i
    kalimat += " "
  return kalimat

df['Abstrak']  = df['Abstrak'].apply(join_kata)
text = df['Abstrak'] 
df['Abstrak'] 

0      sistem informasi akademik siakad sistem inform...
1      jalan koneksi jaring komputer lancar ganggu ha...
2      web server perangkat lunak server fungsi terim...
3      jadwal kuliah guru kompleks masalah variabel t...
4      iring kembang teknologi dunia muncul teknologi...
                             ...                        
800    investasi saham milik resiko rugi gera harga s...
801    information retrieval ir ambil informasi simpa...
802    klasifikasi citra proses kelompok piksel citra...
803    identifikasi atribut pejal kaki salah teliti k...
804    topik deteksi objek tarik perhati kembang tekn...
Name: Abstrak, Length: 805, dtype: object

2. Proses Representasi Data Menggunakan Algoritma TF-IDF


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# )
tfidf_vectorizer = TfidfVectorizer()
tfidf_separate = tfidf_vectorizer.fit_transform(text)

df_tfidf = pd.DataFrame(
    tfidf_separate.toarray(), columns=tfidf_vectorizer.get_feature_names_out(), index=df.index
)
X = df_tfidf.values
df_tfidf

,aalysis,aam,abad,abadi,abai,abdi,ability,abjad,absah,absensi,...,zara,zat,zcz,zf,zona,zone,zoning,zoom,zucara,zungu
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
801,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
802,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
803,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 3. Proses Splitting Data

Proses splitting data ini adalah proses yang membagi data menjadi 2 yaitu data testing dan data training. Data testing yang digunakan pada proses ini yaitu 20% untuk data testing dan 80% untuk data training

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( 
    X, Y, test_size = 0.2, random_state = 100)

y_train

array(['KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'RPL', 'KOMPUTASI',
       'RPL', 'KOMPUTASI', 'RPL', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'RPL',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'RPL',
       'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'RPL',
       'KOMPUTASI', 'KOMPUTASI', 'RPL', 'RPL', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI',
       'RPL', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI',
       'KOMPUTASI', 'KOMPUTASI', 'KOMPUTASI', 'RPL', 'KOMPUTASI',
       'KOMPUTA

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components =10)

X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

pca.explained_variance_ratio_

array([0.02237301, 0.01606353, 0.0150192 , 0.01272603, 0.01076333,
       0.00946327, 0.00859324, 0.00816849, 0.00797871, 0.00742421])

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
from sklearn.metrics import accuracy_score
y_pred = classifier.predict(X_test)
print (" KNN Accuracy : ",
    accuracy_score(y_test,y_pred)*100)

 KNN Accuracy :  81.98757763975155
